In [3]:
import os
import datetime
import pandas as pd
import tushare as ts
import backtrader as bt

### 使用 pandas 从本地数据源加载数据

In [31]:

# 使用 pandas 从本地数据源加载数据
# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.data0_close
        print('切片访问close前10个值', self.data0_close.array[:10])
        print('切片访问close后3个值', self.data0_close.array[-3:])
        print('切片访问close后3个值', self.data0_close.get(ago=-1, size=3))
        print('切片访问open后3个值', self.data0_open.get(ago=-1, size=3))
        print('close0', self.data0_close[0], 'close1', self.data0_close[1], 'close-1', self.data0_close[-1])

        print()

    def next(self):
        # 记录收盘价
        # self.log('Close, %.2f' % self.dataclose[0])
        print('open:',self.data0_open[0],
              'high:',self.data0_high[0],
              'low:',self.data0_low[0],
              'close:',self.data0_close[0],
              )
        pass


if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000
    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)

    # 获取当前运行脚本所在目录
    modpath = os.path.abspath('.')
    # 拼接加载路径
    # file_path = os.path.join('../../datas/orcl-1995-2014.txt')
    file_path = os.path.join('./fd_data/600016.SH.csv')
    date_start="20001219"
    date_end="20001222"
    format="%Y%m%d"

    # 日期格式转换
    dt_start = datetime.datetime.strptime(date_start, format)
    dt_end = datetime.datetime.strptime(date_end, format)

    # 判断文件是否存在
    if not os.path.exists(file_path):
        print("数据源文件未找到！" + file_path)
        raise Exception("数据源文件未找到！" + file_path)

    # 将csv文件转为pandas.dataframe
    df = pd.read_csv(
        filepath_or_buffer=file_path,
        # sep=',',  #分隔符
        # nrows=500,  #读取行数
        # index_col=['Date'],  #设置行索引
        # parse_dates=['Date'],  #解析时间
        # date_parser=lambda x: pd.to_datetime(x, format=format),  #时间解析的格式
        # usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'],  #使用的列
    )
    # 按日期先后排序
    df.sort_values(by=["trade_date"], ascending=True, inplace=True)
    # 将日期列，设置成index
    df.index = pd.to_datetime(df.trade_date, format=format)
    # 截取时间段内样本数据
    df = df[dt_start:dt_end]
    # 增加一列openinterest
    df['openinterest'] = 0.00
    # 取出特定的列
    df = df[['open', 'high', 'low', 'close', 'vol', 'openinterest']]
    # 列名修改成指定的
    df.rename(columns={"vol": "volume"}, inplace=True)

    # print(df.info())
    # print(df.head())
    # 使用pandas数据源创建交易数据集
    data = bt.feeds.PandasData(dataname=df, fromdate=dt_start, todate=dt_end)

    # 加载交易数据
    cerebro.adddata(data)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

    print('data', data)
    # os.system("pause")


组合期初资金: 100000.00
切片访问close前10个值 array('d', [18.56, 18.1, 18.09, 17.66])
切片访问close后3个值 array('d', [18.1, 18.09, 17.66])
切片访问close后3个值 array('d', [18.56, 18.1, 18.09])
切片访问open后3个值 array('d', [20.0, 18.47, 18.18])
close0 17.66 close1 18.56 close-1 18.09

open: 20.0 high: 21.0 low: 18.5 close: 18.56
open: 18.47 high: 18.47 low: 17.91 close: 18.1
open: 18.18 high: 18.48 low: 18.0 close: 18.09
open: 18.1 high: 18.17 low: 17.6 close: 17.66
组合期末资金: 100000.00
data <backtrader.feeds.pandafeed.PandasData object at 0x000001BF60749EB0>


### 使用bt.feeds.GenericCSVData 加载本地csv文件数据

In [59]:

# 使用bt.feeds.GenericCSVData 加载本地csv文件数据
# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.data0_close
        print('切片访问close前10个值', self.data0_close.array[:10])
        print('切片访问close后3个值', self.data0_close.array[-3:])
        print('切片访问close后3个值', self.data0_close.get(ago=-1, size=3))
        print('切片访问open后3个值', self.data0_open.get(ago=-1, size=3))
        print('close0', self.data0_close[0], 'close1', self.data0_close[1], 'close-1', self.data0_close[-1])
        print()

    def next(self):
        # 记录收盘价
        # self.log('Close, %.2f' % self.dataclose[0])
        print('open:',self.data_open[0],
              'high:',self.data_high[0],
              'low:',self.data_low[0],
              'close:',self.data_close[0],
              )
        pass


if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000
    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)

    # 获取当前运行脚本所在目录
    modpath = os.path.abspath('.')
    # 拼接加载路径
    # file_path = os.path.join('../../datas/orcl-1995-2014.txt')
    file_path = os.path.join('./fd_data/600016.SH.csv')
    date_start="20001219"
    date_end="20001229"
    format="%Y%m%d"

    # 使用bt.feeds.GenericCSVData 加载本地csv文件数据
    # ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
    # 600016.SH,20001219,20.0,21.0,18.5,18.56,11.8,6.76,57.29,1563524.56,3058872.084

    # 判断文件是否存在
    if not os.path.exists(file_path):
        print("数据源文件未找到！" + file_path)
        raise Exception("数据源文件未找到！" + file_path)

    # 使用bt.feeds.GenericCSVData 加载本地csv文件数据
    data = bt.feeds.GenericCSVData(
        dataname=file_path,
        fromdate=datetime.datetime.strptime(date_start, format),  #日期格式转换
        todate=datetime.datetime.strptime(date_end, format),  #日期格式转换
        nullvalue=0.0,
        dtformat=format,
        datetime=1,
        high=3,
        low=4,
        open=2,
        close=5,
        volume=9,
        openinterest=-1
    )

    # 加载交易数据
    cerebro.adddata(data)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

    print('data', data)
    # os.system("pause")

组合期初资金: 100000.00
切片访问close前10个值 array('d', [18.56, 18.1, 18.09, 17.66, 17.2, 17.32, 17.11, 17.08])
切片访问close后3个值 array('d', [17.32, 17.11, 17.08])
切片访问close后3个值 array('d', [17.2, 17.32, 17.11])
切片访问open后3个值 array('d', [17.6, 17.2, 17.35])
close0 17.08 close1 18.56 close-1 17.11

open: 20.0 high: 21.0 low: 18.5 close: 18.56
open: 18.47 high: 18.47 low: 17.91 close: 18.1
open: 18.18 high: 18.48 low: 18.0 close: 18.09
open: 18.1 high: 18.17 low: 17.6 close: 17.66
open: 17.6 high: 17.62 low: 17.01 close: 17.2
open: 17.2 high: 17.5 low: 17.18 close: 17.32
open: 17.35 high: 17.38 low: 17.05 close: 17.11
open: 17.1 high: 17.19 low: 17.02 close: 17.08
组合期末资金: 100000.00
data <backtrader.feeds.csvgeneric.GenericCSVData object at 0x000001BF60A51EE0>


### 使用tushare 从网络加载数据源

In [57]:

# 使用tushare 从网络加载数据源
def get_tushare_online_daily_data(
        code="000001.SZ",
        date_start="20190101",
        date_end="20191231",
        format="%Y%m%d", ):
    """使用 tushare 加载 数据返回BT Data
    """
    # %Y%m%d 日期格式的字符串
    dt_start = datetime.datetime.strptime(date_start, format).strftime("%Y%m%d")
    dt_end = datetime.datetime.strptime(date_end, format).strftime("%Y%m%d")
    # TOKEN = '341d66d4586929fa56f3f987e6c0d5bd23fb2a88f5a48b83904d134b'
    TOKEN = 'cd9e075edc5e5106a21a9c38b42a4e92744e08e91405c977e52e69ff'

    ts.set_token(TOKEN)
    pro = ts.pro_api()

    try:
        # 加载数据
        df = pro.daily(ts_code=code, start_date=dt_start, end_date=dt_end)
        df.sort_values(by=["trade_date"], ascending=True,
                       inplace=True)  # 按日期先后排序
        df.reset_index(inplace=True, drop=True)

        # 开始数据清洗：
        # 按日期先后排序
        df.sort_values(by=["trade_date"], ascending=True, inplace=True)
        # 将日期列，设置成index
        df.index = pd.to_datetime(df.trade_date, format='%Y%m%d')
        # 增加一列openinterest
        df['openinterest'] = 0.00
        # 取出特定的列
        df = df[['open', 'high', 'low', 'close', 'vol', 'openinterest']]
        # 列名修改成指定的
        df.rename(columns={"vol": "volume"}, inplace=True)

        print(df.shape[0])
        # print(df.info())
        print(df.head())
        # 使用pandas数据源创建交易数据集
        data = bt.feeds.PandasData(dataname=df)

        return data
    except Exception as err:
        print("下载{0}完毕失败！")
        print("失败原因 = " + str(err))

# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.data0_close
        print('切片访问close前10个值', self.data0_close.array[:10])
        print('切片访问close后3个值', self.data0_close.array[-3:])
        print('切片访问close后3个值', self.data0_close.get(ago=-1, size=3))
        print('切片访问open后3个值', self.data0_open.get(ago=-1, size=3))
        print('close0', self.data0_close[0], 'close1', self.data0_close[1], 'close-1', self.data0_close[-1])
        print()

    def next(self):
        # 记录收盘价
        # self.log('Close, %.2f' % self.dataclose[0])
        print('open:',self.data_open[0],
              'high:',self.data_high[0],
              'low:',self.data_low[0],
              'close:',self.data_close[0],
              )
        pass

if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000
    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)


    code="000001.SZ"
    date_start="20190101"
    date_end="20191231"
    format="%Y%m%d"

    # 使用tushare 从网络加载数据源
    data = get_tushare_online_daily_data(code,date_start,date_end,format)

    # 加载交易数据
    cerebro.adddata(data)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

    print('data', data)
    # os.system("pause")

244
            open  high   low  close      volume  openinterest
trade_date                                                   
2019-01-02  9.39  9.42  9.16   9.19   539386.32           0.0
2019-01-03  9.18  9.33  9.15   9.28   415537.95           0.0
2019-01-04  9.24  9.82  9.22   9.75  1481159.06           0.0
2019-01-07  9.84  9.85  9.63   9.74   865687.66           0.0
2019-01-08  9.73  9.74  9.62   9.66   402388.11           0.0
组合期初资金: 100000.00
切片访问close前10个值 array('d', [9.19, 9.28, 9.75, 9.74, 9.66, 9.94, 10.1, 10.2, 10.11, 10.24])
切片访问close后3个值 array('d', [16.63, 16.57, 16.45])
切片访问close后3个值 array('d', [16.47, 16.63, 16.57])
切片访问open后3个值 array('d', [16.34, 16.53, 16.46])
close0 16.45 close1 9.19 close-1 16.57

open: 9.39 high: 9.42 low: 9.16 close: 9.19
open: 9.18 high: 9.33 low: 9.15 close: 9.28
open: 9.24 high: 9.82 low: 9.22 close: 9.75
open: 9.84 high: 9.85 low: 9.63 close: 9.74
open: 9.73 high: 9.74 low: 9.62 close: 9.66
open: 9.74 high: 10.08 low: 9.7 close: 9.94
open: 9

In [5]:
import akshare
from datetime import datetime

# 使用 akshare 从网络加载数据源
def get_akshare_online_daily_data(
        code="000001.SZ",
        date_start="20190101",
        date_end="20191231",
        format="%Y%m%d", ):
    """利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
    """
    try:
        # 利用 AKShare 获取股票的后复权数据，这里只获取前 6 列
        df = akshare.stock_zh_a_hist(symbol="000001",start_date=date_start, end_date=date_end, adjust="hfq").iloc[:, :6]
        # 处理字段命名，以符合 Backtrader 的要求
        df.columns = ['date', 'open', 'close', 'high', 'low', 'volume', ]
        # 列名修改成指定的
        df.rename(columns={"vol": "volume"}, inplace=True)

        print(df.shape[0])
        # print(df.info())
        print(df.head())
        # 使用pandas数据源创建交易数据集
        data = bt.feeds.PandasData(dataname=df)

        return data
    except Exception as err:
        print("下载{0}完毕失败！")
        print("失败原因 = " + str(err))

# 创建策略继承bt.Strategy
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        # 记录策略的执行日志
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 保存收盘价的引用
        self.dataclose = self.data0_close
        print('切片访问close前10个值', self.data0_close.array[:10])
        print('切片访问close后3个值', self.data0_close.array[-3:])
        print('切片访问close后3个值', self.data0_close.get(ago=-1, size=3))
        print('切片访问open后3个值', self.data0_open.get(ago=-1, size=3))
        print('close0', self.data0_close[0], 'close1', self.data0_close[1], 'close-1', self.data0_close[-1])
        print()

    def next(self):
        # 记录收盘价
        # self.log('Close, %.2f' % self.dataclose[0])
        print('open:',self.data_open[0],
              'high:',self.data_high[0],
              'low:',self.data_low[0],
              'close:',self.data_close[0],
              )
        pass

if __name__ == '__main__':
    # 创建Cerebro引擎
    cerebro = bt.Cerebro()
    # Cerebro引擎在后台创建broker(经纪人)，系统默认资金量为10000
    # 为Cerebro引擎添加策略
    cerebro.addstrategy(TestStrategy)


    code="000001.SZ"
    date_start="20190101"
    date_end="20191231"
    format="%Y%m%d"

    # 使用 akshare 从网络加载数据源
    data = get_akshare_online_daily_data(code,date_start,date_end,format)

    # 加载交易数据
    cerebro.adddata(data)

    # 设置投资金额100000.0
    cerebro.broker.setcash(100000.0)
    # 引擎运行前打印期出资金
    print('组合期初资金: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    # 引擎运行后打期末资金
    print('组合期末资金: %.2f' % cerebro.broker.getvalue())

    print('data', data)
    # os.system("pause")

    '''
*注意：**这里我在学习使用的时候，因为对pandas包不熟悉，浪费了很多时间。这里说明一下pandas读取Excel文件赋值到pandas.dataframe变量，作为Data Feed的时候，需要注意的地方：

pandas.read_excel()函数读取日期需要进行日期的解析
读取到的DataFrame类型的数据是没有设置index的，需要手动设置。并且不能为整数类型的时间戳，否则会报错 AttributeError: ‘numpy.int64’ object has no attribute ‘to_pydatetime’. 也不能为字符串类型的数据，否则会报错 AttributeError: ‘str’ object has no attribute ‘to_pydatetime’.
读取到的DataFrame中index有可能是降序排列的（离当前日期最近的一天在dataframe中第一行，最小的日期在最后一行），这在backtrader中不符合逻辑，需要进行重新排列
    '''

244
         date     open    close     high      low   volume
0  2019-01-02  1606.73  1574.22  1611.60  1569.35   539386
1  2019-01-03  1572.60  1588.85  1596.98  1567.72   415538
2  2019-01-04  1582.35  1665.24  1676.62  1579.10  1481159
3  2019-01-07  1679.87  1663.61  1681.49  1645.74   865688
4  2019-01-08  1661.99  1650.61  1663.61  1644.11   402388
组合期初资金: 100000.00


AttributeError: 'int' object has no attribute 'to_pydatetime'